# Noise bug

Author: Louise

Using the notebooks done by Daniele and Gabriele, I try to reproduce the same.

In [ ]:
#Imports
%matplotlib inline
import numpy as np
import pylab as pl #plots
import healpy as hp #maps
import random #random numb gen

import qubic #qubic soft
from qubic import SpectroImLib as si
from pysm.nominal import models
from pysimulators import FitsArray 
import os #used to call environment variables

In [ ]:
inst_dict_fn = 'TD_spectroimaging.dict'

# Read dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file(inst_dict_fn)

# kmax
print("kmax = %1i" % d['synthbeam_kmax'])

# Read noise
nep = d['detector_nep']
print('Detector NEP = %1.2e W/sqrt(Hz)' % nep)

# Define sky configuration
sky_config = {'cmb': models('c1', d['nside'])}

# Define number of sub-bands
nf_sub = d['nf_sub']
print('nf_sub: ', nf_sub)

# Define output map filename
out_map = './'

In [ ]:
# Input sky map
Qubic_sky = si.Qubic_sky(sky_config, d)
x0 = Qubic_sky.get_simple_sky_map()

print(x0.shape)

hp.mollview(x0[:, 0])

In [ ]:
# Pointing strategy
p = qubic.get_pointing(d)
print('=== Pointing DONE! ===')

# Scene
s = qubic.QubicScene(d)

In [ ]:
# Make a mono detector instrument
def select_detector(instrument, FPindex):
    '''
    This function allows one to select a single detector for the simulation

    INPUTS
    instrument - qubic.instrument.QubicMultibandInstrument - the MultiInstrument object
    FPindex - INT - detector index ("0" is 594)

    OUTPUTS
    out -qubic.instrument.QubicMultibandInstrument -_The updated MultiInstrument object
    '''

    a = int(np.where(instrument.detector.index == FPindex)[0])
    print(instrument.detector.index)
    print(FPindex, a, instrument.detector.index[a])
    mono_instrument = instrument[a]
    
    out = mono_instrument
    return out

q = qubic.QubicInstrument(d)

# Detector at the center of the FP
q_mono = q[0]
q_mono.detector.center = np.array([[1e-5, 1e-5, -0.3]])

# Another detector
# FPindex = 594
# q_mono = select_detector(q, FPindex)



In [ ]:
# Acquisition 
a = qubic.QubicAcquisition(q_mono, p, s, d)

#TOD
tod, maps_convolved_useless = a.get_observation(x0, noiseless=d['noiseless'], convolution = True)
del(maps_convolved_useless)
  
print(tod.shape)

pl.plot(tod[0,:]);
pl.xlabel('time (s)');
pl.ylabel('data (W)');
pl.title('QUBIC TOD');


In [ ]:
# sigma measured and theo
period = d['period']
sigma = np.std(tod[0, :])
print('sigma measured = %1.3e W, sigma theo = %1.3e W' %(sigma, nep/np.sqrt(2)/np.sqrt(period)))

In [ ]:
# Get coverage map and hitmap
# i.e. how many times were seen each pixel of the sky (not trivial because of the synthetic beam)
cov = a.get_coverage()

hp.mollview(cov)

hitmap = a.get_hitmap()
hp.mollview(hitmap)

npix_patch = len(cov[cov != 0.])
print('npix_patch:', npix_patch)
print(np.sum(hitmap))

In [ ]:
# Check that if the detector is at the center, hitmap and cov are equal
for c, b in zip(cov, hitmap):
#     print(i, cov)
    if c != 0. or b != 0:
        print(c, b)

In [ ]:
# QUBIC map making
maps_recon, nit, error = a.tod2map(tod, d, cov=None)

print('The shape of the reconstructed maps is (nf_recon, #pixels, #stokes) :', maps_recon.shape)
print('{} iterations were needed to get an error of {}'.format(nit, error))    

The conversion below is wrong, the map obtained after the map making is in µK but can not be converted to Watt using directly the Planck law. There are many conversions in the H operator that we should take into account. 

In [ ]:
def nep2net(nep, freq, bandwidth, temp):
    h = 6.62607004e-34
    k = 1.38064852e-23
    x = h*freq/k/temp
    dPdT = (2*k*bandwidth) * (x**2*np.exp(x)) /(np.exp(x)-1)**2
    net = nep / dPdT
    return net

In [ ]:
# Convert to NET
nu0 = d['filter_nu']
bw = nu0 * d['filter_relative_bandwidth']
temp = d['temperature']
net = nep2net(nep, nu0, bw, temp)

print('Center frequency = %3.0f GHz' % (nu0 / 1e9))
print('Bandwidth = %3.0f GHz' % (bw / 1e9))
print('Integration time = %1.3f s' % period)
print('Detector NET = %1.2e K x sqrt(s)' % net)

In [ ]:
# Select pixels with good coverage
threshold = 0.1
okpix = cov[:] > (threshold * np.max(cov[:]))

themap = np.copy(maps_recon)
# Set the uncovered pixels to UNSEEN and select the intensity map
for i in np.arange(3):
    themap[~okpix,i] = hp.UNSEEN #conventional number for grey background
themap_I = themap[:, 0]

Tobs = sum(cov[okpix]) * d['period']

Npix = len(themap_I[okpix])
tau_m = Tobs / Npix #tau medio
print(Tobs, Npix, tau_m)

sigma_th = sigma / np.sqrt(tau_m)
sigma_th_k = nep2net(sigma_th, nu0, bw, temp)
print('sigma theory: ', sigma_th_k, 'K')

sigma_map = np.std(themap[okpix, :]) * 1e-6 # Convert from µK to K
print('sigma in the map:', sigma_map, 'K')


rms_qubic = sigma_map / sigma_th_k
print('rapport:', rms_qubic)

hp.mollview(themap_I, title = 'Data map w/ thereshold');

### Binned map making

This is actually not so easy still because of conversion unit... TOD are in Watt, the final map is in µK but the conversion is not trivial. So this map making can not be compared to the QUBIC one so easily.

In [ ]:
nside = d['nside']
n_pixels = 12 * ( nside** 2)
pix = hp.ang2pix(nside, p[:].galactic[:].T[0], p[:].galactic[:].T[1], lonlat=True)
mappa = np.array([0.] * n_pixels)
hits = np.array([0] * n_pixels)


for count in np.arange(np.size(tod[0])):
    mappa[pix[count]] += tod[0,count]
    hits[pix[count]] += 1

mappa = mappa / hits #averaging data in pixel

In [ ]:
# Select pixels with good coverage
threshold = 0.1 
okpix = hits[:] > (threshold * np.max(hits[:]))
#area defined by RA_center and DEC_center in Pointing section, dictionary
print('threshold = ', threshold)

# Set the uncovered pixels to UNSEEN and select the intensity map
mappa[~okpix] = hp.UNSEEN #conventional number for grey background
mappa_I = mappa[:] #not really needed

Tobs = sum(hits[okpix]) * d['period']
Npix = len(mappa[okpix])
tau_m = Tobs/Npix
sigma_map = sigma / np.sqrt(tau_m)
print('sigma_map = ',sigma_map, 'W')

rms_test = np.std(mappa_I[okpix])/(sigma_map)
print('rms = ', rms_test)

# Study the H operator

Compute the order of magnitude of H.

In [ ]:
# Look at some parameters
q.detector.nep
q.horn.radeff

print(q.optics.components['transmission'])
print(q.detector.efficiency)

s.atmosphere.transmission

q.detector.tau

p.period
q.filter.bandwidth/1e9

q.synthbeam.peak150.solid_angle

In [ ]:
# Compute the factor to go from µK to W/m2/Hz using the Planck Law
# This is done by the function get_unit_conversion_operator(self, nu) in pysimulators/interfaces/healpy/scenes.py

nu = 150e9
h = 6.626e-34
c = 3e8
T = 2.7
k = 1.38e-23
nside = 256
npix = 12 * nside**2
Omega_pix = 4 * np.pi / npix

dT = 200

x = h * nu / (k * T)

y = 2 * Omega_pix * h * nu**3 / c**2

factor = 1e-6 * y * x * np.exp(x) / ((np.exp(x) - 1)**2 * T)
print(factor)



In [ ]:
nhorns = 64
radeff = 4.67e-3 # m
aperture = nhorns * np.pi * radeff**2

bandwidth = 37.5e9 # Hz


Omega_beam150 = np.deg2rad(1.02)**2

A_det = (3e-3)**2 # m^2

Df = 30e-2 # m

Omega_TES = A_det / Df**2

Omega_beam_sec = np.deg2rad(13)**2

rhodet = 0.8

eff_optics = 0.98**17

tau = 0.01

npix_patch = 13000
dT = 200 # µK

H = factor * aperture * bandwidth * (Omega_beam150 / Omega_pix) * nhorns * 0.5 \
    * (Omega_TES / Omega_beam_sec) * rhodet * eff_optics

print( H * dT * npix_patch)

# Fake TOD with a white noise

In [ ]:
tod_fake = np.empty((1, d['npointings']))

for count in np.arange(d['npointings']): #White noise
    tod_fake[0, count] = random.gauss(0, nep/np.sqrt(2)/np.sqrt(tau)) 
    
p = qubic.get_pointing(d)
pl.plot(tod_fake[0,:]);
pl.xlabel('time (s)');
pl.ylabel('data (W)');
pl.title('White Noise TOD');

In [ ]:
map_recon_fake, nit, error = a.tod2map(tod_fake, d)

In [ ]:
map_recon_fake.shape
print(np.std(map_recon_fake[:, 0]))
print(nep)